# IR Practical No.-3
                            -By Aditya Nikam(BE AI & DS)

Write a program Write a program to construct a Bayesian network considering medical data. Use this model to demonstrate the diagnosis of heart patients using the standard Heart Disease Data Set.

In [20]:
!pip install pgmpy


Defaulting to user installation because normal site-packages is not writeable
  Using cached pgmpy-1.0.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached torch-2.9.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached pyro_ppl-1.9.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached pgmpy-1.0.0-py3-none-any.whl (2.0 MB)
Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
Using cached pyro_ppl-1.9.1-py3-none-any.whl (755 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   -------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Predator\\AppData\\Roaming\\Python\\Python312\\site-packages\\torch\\autograd\\functional.py'
Check the permissions.



In [24]:

import pandas as pd
import numpy as np
try:
    from pgmpy.models import DiscreteBayesianNetwork as BN   # for pgmpy ≥ 1.0.0
except ImportError:
    from pgmpy.models import BayesianModel as BN             # fallback for older versions
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination


ModuleNotFoundError: No module named 'pgmpy'

In [25]:
# Load the dataset (keep heart.csv in the same folder)
df = pd.read_csv("heart.csv").replace('?', np.nan).dropna()

# Rename target column if needed
for t in ['target', 'num']:
    if t in df.columns and 'heartdisease' not in df.columns:
        df = df.rename(columns={t: 'heartdisease'})

# Keep only important columns
cols = ['age','sex','trestbps','fbs','exang','restecg','thalach','chol','heartdisease']
df = df[[c for c in cols if c in df.columns]].copy()

df.head()


,age,sex,trestbps,fbs,exang,restecg,thalach,chol,heartdisease
0,52,1,125,0,0,1,168,212,0
1,53,1,140,1,1,0,155,203,0
2,70,1,145,0,1,1,125,174,0
3,61,1,148,0,0,1,161,203,0
4,62,0,138,1,0,1,106,294,0


In [27]:
if 'age' in df:       df['age'] = (df['age'].astype(float) > 50).astype(int)
if 'trestbps' in df:  df['trestbps'] = (df['trestbps'].astype(float) > 130).astype(int)
if 'thalach' in df:   df['thalach'] = (df['thalach'].astype(float) < 150).astype(int)  # 1 = low
if 'chol' in df:      df['chol'] = (df['chol'].astype(float) > 200).astype(int)
df = df.dropna().astype(int)

df.head()


,age,sex,trestbps,fbs,exang,restecg,thalach,chol,heartdisease
0,1,1,0,0,0,1,0,1,0
1,1,1,1,1,1,0,0,1,0
2,1,1,1,0,1,1,1,0,0
3,1,1,1,0,0,1,0,1,0
4,1,0,1,1,0,1,1,1,0


In [29]:
features = [c for c in df.columns if c != 'heartdisease']
model = BN([('heartdisease', f) for f in features])

# Learn CPDs
model.fit(df, estimator=MaximumLikelihoodEstimator)
infer = VariableElimination(model)

print("Learning CPDs using Maximum Likelihood Estimators...\n")
print("Inferencing with Bayesian Network:")


NameError: name 'BN' is not defined

In [31]:
def bin_evidence(evidence):
    b = {}
    for k, v in evidence.items():
        if k == 'age': b[k] = int(float(v) > 50)
        elif k == 'trestbps': b[k] = int(float(v) > 130)
        elif k == 'chol': b[k] = int(float(v) > 200)
        elif k == 'thalach': b[k] = int(float(v) < 150)
        else: b[k] = int(v)
    return b


In [ ]:
print("\n1. Probability of HeartDisease given Age = 28")
print(infer.query(['heartdisease'], evidence=bin_evidence({'age': 28})))

print("\n2. Probability of HeartDisease given Cholesterol = 100")
print(infer.query(['heartdisease'], evidence=bin_evidence({'chol': 100})))
